In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Input, Flatten, Reshape, ConvLSTM2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import sklearn
# fix random seed for reproducibility
tf.random.set_seed(7)
# pip3 install tensorflow numpy pandas scikit-learn

2023-05-10 20:40:47.790180: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 20:40:47.927744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-10 20:40:47.927761: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-10 20:40:48.712155: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [23]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df1 = pd.read_csv("Challenge_2_submission_template.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df1

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,2022-04-10,1.40,NaN
1,2,Chau_Thanh,10.400189,105.331053,SA,T,2022-07-15,1.32,NaN
2,3,Chau_Phu,10.505489,105.203926,SA,D,2022-07-14,1.40,NaN
3,4,Chau_Phu,10.523520,105.138274,WS,D,2022-04-10,1.80,NaN
4,5,Thoai_Son,10.294660,105.248528,SA,T,2022-07-20,2.20,NaN
...,...,...,...,...,...,...,...,...,...
95,96,Chau_Thanh,10.435839,105.132981,SA,D,2022-07-26,1.21,NaN
96,97,Chau_Phu,10.529357,105.147388,WS,T,2022-04-10,2.00,NaN
97,98,Chau_Thanh,10.452537,105.205118,SA,T,2022-07-20,5.50,NaN
98,99,Chau_Thanh,10.394341,105.126836,SA,T,2022-07-14,4.40,NaN


In [12]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df2 = pd.read_csv("Crop_Yield_Data_challenge_2.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df2

,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Rice Yield (kg/ha)
0,Chau_Phu,10.510542,105.248554,SA,T,2022-07-15,3.40,5500
1,Chau_Phu,10.509150,105.265098,SA,T,2022-07-15,2.43,6000
2,Chau_Phu,10.467721,105.192464,SA,D,2022-07-15,1.95,6400
3,Chau_Phu,10.494453,105.241281,SA,T,2022-07-15,4.30,6000
4,Chau_Phu,10.535058,105.252744,SA,D,2022-07-14,3.30,6400
...,...,...,...,...,...,...,...,...
552,Thoai_Son,10.364419,105.164984,WS,T,2022-04-12,7.80,6640
553,Thoai_Son,10.358094,105.189541,WS,T,2022-04-12,2.00,7200
554,Thoai_Son,10.368014,105.238516,WS,T,2022-04-12,6.20,7200
555,Thoai_Son,10.275419,105.234563,WS,T,2022-04-20,3.00,6400


In [ ]:
df.loc[0]

In [19]:
df2.iloc[:,7].min()

5200

In [25]:
borne = [
    (-0.0440750, 1.1379025),
    (-0.0215250, 1.1155175),
    (-0.1473925, 1.13914),
    (0.04604250, 1.2140225),
    (21762.0000, 56916.0),
    (293.0, 48423.0)
]
mini = np.array([
    -0.0440750,
    -0.0215250,
    -0.1473925,
    0.04604250,
    21762.0000,
    293.0,
])
multi = np.array([
    1.1819775, 
    1.1370425, 
    1.2865325, 
    1.16798, 
    35154.0, 
    48130.0
])

In [26]:
x1 = []
y1 = []
for i in range(len(df1)):
    with open(f"inputs_lstm_val/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x1.append(arr)
    if df1.loc[i, "Season(SA = Summer Autumn, WS = Winter Spring)"] == "SA":
        y1.append([1, 0])
    else:
        y1.append([0, 1])

In [ ]:
x2 = []
y2 = []
for i in range(len(df2)):
    with open(f"inputs_lstm/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x2.append(arr)
    if df2.loc[i, "Season(SA = Summer Autumn, WS = Winter Spring)"] == "SA":
        y2.append([1, 0])
    else:
        y2.append([0, 1])

In [ ]:
x2 = []
y2 = []
for i in range(len(df2)):
    with open(f"inputs_lstm/{i}.npy", "rb") as f:
        arr = np.load(f)
        arr = np.transpose(arr, (0, 2, 3, 1))
        arr = (arr-mini)/multi
        x2.append(arr)
    y2.append([(df2.loc[i, "Rice Yield (kg/ha)"]-5200)/(8000-52000)])

In [ ]:
x2 = pad_sequences(x2, dtype="float", maxlen=32)
y2 = np.array(y2)

In [27]:
x1 = pad_sequences(x1, dtype="float", maxlen=32)
y1 = np.array(y1)

In [ ]:
model = load_model("models_finaux/lstm_profond/68-0.98.h5")

In [ ]:
model = load_model("models_finaux/petit_lstm/128-0.94.h5")

In [34]:
model = load_model("models_finaux/petit_lstm_g/14-1.00.h5")

In [35]:
y1_predict = model.predict(x1)
(y1.argmax(axis=1)==y1_predict.argmax(axis=1)).sum()

2023-05-10 21:19:31.422442: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 768000000 exceeds 10% of free system memory.


4/4 [==============================] - 11s 2s/step


100

In [ ]:
y2_predict = model.predict(x2[:50])
(y2[:50].argmax(axis=1)==y2_predict.argmax(axis=1)).sum()

In [ ]:
x = pad_sequences(x1+x2)
y = np.array(y1+y2)
x1 = pad_sequences(x1)
y1 = np.array(y1)
# x, y = shuffle(x, y)

In [ ]:
class_weight = {
    0: 1/np.count_nonzero(y[:, 0]==1),
    1: 1/np.count_nonzero(y[:, 1]==1)
}

In [ ]:
np.count_nonzero(y[:, 0]==1)

In [ ]:
np.count_nonzero(y[:, 1]==1)

In [ ]:
"""
model = Sequential()
model.add(Input(shape=(None,100, 100, 6)))
model.add(ConvLSTM2D(filters=3, kernel_size=3, return_sequences=False))

pretrained = ResNet50(
    include_top=False, 
    input_shape=model.output_shape[1:]
)
for layer in pretrained.layers:
    layer.trainable = False
    
model.add(pretrained)
model.add(Flatten())
model.add(Dense(10))
model.add(Dense(2, activation="softmax"))
"""
model = load_model()
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# import os
# checkpoint = ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5', monitor="val_loss", mode="min", save_best_only=True, verbose=1)
checkpoint = ModelCheckpoint(
    filepath='model/{epoch:02d}-{val_acc:.2f}.h5', 
    monitor="val_acc", mode="max", 
    save_best_only=True, verbose=1
)
callbacks = [checkpoint]

In [ ]:
%%time
history = model.fit(
    x, y, 
    batch_size=16, shuffle=True, 
    validation_data=(x1, y1)
    # class_weight=class_weight,
    epochs=1_000, callbacks=callbacks
)

In [ ]:
import time
print(time.asctime())

# New model

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 98, 98, 3)         984       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 2048)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 32)                262176    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                        

In [7]:
model.input_shape

(None, None, 100, 100, 6)

In [32]:
model_reg = Sequential()
model = load_model("models_finaux/petit_lstm_g/07-0.99.h5")
model_reg.add(Input(shape=(None, 100, 100, 6)))
for layer in model.layers[:-1]:
    layer.trainable = False
    model_reg.add(layer)
model_reg.add(Dense(16))
model_reg.add(Dense(16))
model_reg.add(Dense(1, activation="sigmoid"))
model_reg.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 98, 98, 3)         984       
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 2048)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 32)                262176    
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                      

In [29]:
model_reg.compile(
    optimizer='sgd',
    loss='mse',
    metrics=['mse']
)

In [33]:
model_reg.predict(x1)

2023-05-10 20:58:52.488801: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 768000000 exceeds 10% of free system memory.


4/4 [==============================] - 11s 2s/step


array([[0.16817054],
       [0.00199232],
       [0.01300677],
       [0.20708033],
       [0.0019566 ],
       [0.22542115],
       [0.00235839],
       [0.00189835],
       [0.14358068],
       [0.2140939 ],
       [0.00943371],
       [0.11490636],
       [0.02068991],
       [0.00672141],
       [0.0049293 ],
       [0.2287198 ],
       [0.00212733],
       [0.00202162],
       [0.0020001 ],
       [0.23512319],
       [0.20528927],
       [0.02560289],
       [0.22145475],
       [0.00202188],
       [0.23406598],
       [0.22076684],
       [0.22507255],
       [0.00329075],
       [0.00199489],
       [0.02778488],
       [0.00194246],
       [0.00244069],
       [0.26091856],
       [0.23519224],
       [0.16154517],
       [0.23242725],
       [0.0027766 ],
       [0.00346084],
       [0.2237436 ],
       [0.01371218],
       [0.05653216],
       [0.00783963],
       [0.00573497],
       [0.22214223],
       [0.00199268],
       [0.00193679],
       [0.00247258],
       [0.135